## Quick start: Subscription Example

There are three levels in the dxfeed package. The lowest is the C API library, the highest is Python wrapper classes.  Cython level in the middle aims to connect these two. Here we are going to look into Python level.

Python level, in its turn, mainly consists of three class types:

1. Endpoint
2. Subscription
3. EventHandler


The **Endpoint** is responsible for connection management and  creating dependent classes, for
example Subscription. One Endpoint may have several different
Subscriptions, but each Subscription is related to one Endpoint.

**Subscription** class sets the type of subscription (stream or timed), the type of
events (e.g. Trade, Candle), etc.

After you specified the data you want to receive, you have to specify
how to process upcoming events. This is where the **EventHandler** class and
its children come into play. Every time an event arrives Cython event
listener will call ``self.update(event)`` method. You have to inherit
from the EventHandler class and redefine the update method. Or you may
use DefaultHandler which stores upcoming data in deque of the length
100k.

### Import package

In [ ]:
import dxfeed as dx
from datetime import datetime  # for timed subscription

### Configure and create connection with Endpoint class
Create instance of Endpoint class which will connect provided address. 

In [ ]:
endpoint = dx.Endpoint('demo.dxfeed.com:7300')

Endpoint instance contains information about the connection, e.g. connection address or status

In [ ]:
print(f'Connected address: {endpoint.address}')
print(f'Connection status: {endpoint.connection_status}')

### Configure and create subscription
You should specify event type. For timed subscription (conflated stream) you should also provide time to start subscription from.

In [ ]:
trade_sub = endpoint.create_subscription('Trade')

**Set event handler** - class that process incoming events. Here we use default one

In [ ]:
trade_handler = dx.DefaultHandler()
trade_sub.set_event_handler(trade_handler);

**Add tikers** you want to recieve events for

In [ ]:
trade_sub = trade_sub.add_symbols(['C', 'IBM'])

For timed subscription you should provide either datetime object or string. String might be incomlete, in this case you will get warning with how your provided date parsed automatically

In [ ]:
tns_sub = endpoint.create_subscription('TimeAndSale', date_time=datetime.now()) \
                  .add_symbols(['AMZN'])

In [ ]:
candle_sub = endpoint.create_subscription('Candle', date_time='2020-04-16 13:05')
candle_sub = candle_sub.add_symbols(['AAPL', 'MSFT'])

**Note**
Two previous subscriptions attached DefaultHandler implicitly. To retrieve instances just call `get_event_handler()` method.

In [ ]:
tns_handler = tns_sub.get_event_handler()
candle_handler = candle_sub.get_event_handler()

#### Subscription instance properties

In [ ]:
print(f'TimeAndSale subscription event type: {tns_sub.event_type}')
print(f'Candle subscription symbols: {candle_sub.symbols}')

### Access data from DefaultHandler instance
You can get colums, list or dataframe.
You are also allowed to write handler that stores no data.

In [ ]:
print(f'Trade columns: {trade_handler.columns}')
print(f'Candle columns: {candle_handler.columns}')

In [ ]:
candle_handler.get_list()

In [ ]:
candle_handler.get_dataframe()

### Close subscription

In [ ]:
trade_sub.close_subscription()
tns_sub.close_subscription()
candle_sub.close_subscription()

### Close connection

In [ ]:
endpoint.close_connection()
print(f'Connection status: {endpoint.connection_status}')